In [1]:
# install libraries
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git" --no-cache-dir
!pip install --no-deps "trl<0.9.0" peft accelerate bitsandbytes xformers datasets -q

  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-install-ft7haio7/unsloth_5fe91994ff844df99938bc1466ebd0e2
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-install-ft7haio7/unsloth_5fe91994ff844df99938bc1466ebd0e2
  Resolved https://github.com/unslothai/unsloth.git to commit 8c432a9d52a735f66fe1d3bcdfc0b2b0dc271a2e
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 MB 50.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 49.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.1/162.1 kB 112.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 145.8/145.8 kB 224.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.3/128.3 kB 240.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 254.1 MB/s eta 0:

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.2/245.2 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.5/31.5 MB 15.1 MB/s eta 0:00:00


In [1]:
# import libraries
from unsloth import FastLanguageModel
from transformers import TrainingArguments
from trl import SFTTrainer
from datasets import load_dataset

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


    PyTorch 2.7.0+cu126 with CUDA 1206 (you have 2.6.0+cu124)
    Python  3.11.12 (you have 3.11.12)
  Please reinstall xformers (see https://github.com/facebookresearch/xformers#installing-xformers)
  Memory-efficient attention, SwiGLU, sparse and more won't be available.
  Set XFORMERS_MORE_DETAILS=1 for more details


🦥 Unsloth Zoo will now patch everything to make training faster!


In [2]:
dataset = load_dataset("QuyenAnhDE/Diseases_Symptoms", split="train")
dataset

README.md:   0%|          | 0.00/381 [00:00<?, ?B/s]

Repo card metadata block was not found. Setting CardData to empty.


Diseases_Symptoms.csv:   0%|          | 0.00/107k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/400 [00:00<?, ? examples/s]

Dataset({
    features: ['Code', 'Name', 'Symptoms', 'Treatments'],
    num_rows: 400
})

In [3]:
def format_row(row):
    return {
        "text": f"""### Instruction:
Given the disease name and its symptoms, provide the diagnosis code and suggested treatments.

### Input:
Name: {row['Name']}
Symptoms: {row['Symptoms']}

### Response:
Code: {row['Code']}
Treatments: {row['Treatments']}
"""
    }

In [4]:
formatted_dataset = dataset.map(format_row)
formatted_dataset = formatted_dataset.remove_columns(['Code', 'Name', 'Symptoms', 'Treatments'])
formatted_dataset

Map:   0%|          | 0/400 [00:00<?, ? examples/s]

Dataset({
    features: ['text'],
    num_rows: 400
})

In [5]:
formatted_dataset[0]

{'text': '### Instruction:\nGiven the disease name and its symptoms, provide the diagnosis code and suggested treatments.\n\n### Input:\nName: Panic disorder\nSymptoms: Palpitations, Sweating, Trembling, Shortness of breath, Fear of losing control, Dizziness\n\n### Response:\nCode: 1\nTreatments: Antidepressant medications, Cognitive Behavioral Therapy, Relaxation Techniques\n'}

In [7]:
from unsloth import FastLanguageModel, UnslothTrainer, UnslothTrainingArguments
import torch

# step-1 load tokenizer and model
base_model = "microsoft/phi-2"
max_seq_length = 2048
load_in_4bit = True
dtype = None

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = base_model,
    max_seq_length = max_seq_length,
    load_in_4bit = load_in_4bit,
    dtype = dtype,
    trust_remote_code = True,
    device_map = "auto",
)

# step-2 apply lora adapters
model = FastLanguageModel.get_peft_model(
    model,
    r=16,
    target_modules=[
        "q_proj", "k_proj", "v_proj", "o_proj",
        "gate_proj", "up_proj", "down_proj"
    ],
    lora_alpha=16,
    lora_dropout=0,
    bias="none",
    use_gradient_checkpointing="unsloth",
    random_state=3407,
    use_rslora=False,
    loft_config=None
)

# step-3. Initialize the UnslothTrainer
trainer = UnslothTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=formatted_dataset,
    dataset_text_field="text",  # Adjust based on your dataset's column name
    max_seq_length=max_seq_length,
    args=UnslothTrainingArguments(
        per_device_train_batch_size=4,
        gradient_accumulation_steps=4,
        learning_rate=3e-4,
        embedding_learning_rate=5e-6,  # Smaller for lm_head/embed_tokens
        max_steps=100,  # Adjust based on dataset size
        warmup_steps=10,
        fp16=not torch.cuda.is_bf16_supported(),
        bf16=torch.cuda.is_bf16_supported(),
        logging_steps=1,
        output_dir="./phi2-finetuned-unsloth",
        optim="adamw_8bit",
        seed=3407,
    ),
)

# 6. Start training
trainer.train()

Unsloth: WARNING `trust_remote_code` is True.
Are you certain you want to do remote code execution?
==((====))==  Unsloth 2025.5.10: Fast Phi patching. Transformers: 4.52.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = None. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: Phi does not support SDPA - switching to eager!


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

microsoft/phi-2 does not have a padding token! Will use pad_token = <|endoftext|>.
Unsloth: Making `model.base_model.model.model` require gradients


Map:   0%|          | 0/400 [00:00<?, ? examples/s]

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


...

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: nelliparthi123 (nelliparthi123-appneural-) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
1,1.820000
2,1.900900
3,1.902700
4,1.907700
5,1.825400
6,1.827400
7,1.781100
8,1.842000
9,1.747800
10,1.566700


TrainOutput(global_step=100, training_loss=0.8725003123283386, metrics={'train_runtime': 547.1362, 'train_samples_per_second': 2.924, 'train_steps_per_second': 0.183, 'total_flos': 3526376469258240.0, 'train_loss': 0.8725003123283386, 'epoch': 4.0})

In [11]:
from peft import PeftModel

# Merge LoRA with base model
merged_model = model.merge_and_unload()  # merges LoRA into base model

# Save full merged model (with tokenizer)
merged_model.save_pretrained("phi2-merged")
tokenizer.save_pretrained("phi2-merged")

/usr/local/lib/python3.11/dist-packages/peft/tuners/lora/bnb.py:351: UserWarning: Merge lora module to 4-bit linear may get different generations due to rounding errors.
  warnings.warn(


('phi2-merged/tokenizer_config.json',
 'phi2-merged/special_tokens_map.json',
 'phi2-merged/vocab.json',
 'phi2-merged/merges.txt',
 'phi2-merged/added_tokens.json',
 'phi2-merged/tokenizer.json')

In [12]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
# load original model
original_model = AutoModelForCausalLM.from_pretrained(
    "microsoft/phi-2",
    #trust_remote_code=True,
    device_map="auto",
    torch_dtype = torch.float16,
)
original_tokenizer = AutoTokenizer.from_pretrained("microsoft/phi-2",use_fast=True)
original_tokenizer.pad_token=original_tokenizer.eos_token

# Load fine-tuned Phi-2
finetuned_model = AutoModelForCausalLM.from_pretrained(
    "phi2-merged",  # or wherever you saved your model
    device_map="auto",
    torch_dtype=torch.float16,
)
finetuned_tokenizer = AutoTokenizer.from_pretrained("phi2-merged", use_fast=True)
finetuned_tokenizer.pad_token = finetuned_tokenizer.eos_token

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [13]:
original_pipe = pipeline("text-generation", model=original_model, tokenizer=original_tokenizer)
finetuned_pipe = pipeline("text-generation", model=finetuned_model, tokenizer=finetuned_tokenizer)


Device set to use cuda:0
Device set to use cuda:0


In [14]:
prompt = """### Instruction:
Given the disease name and its symptoms, provide the diagnosis code and suggested treatments.

### Input:
Name: Panic disorder
Symptoms: Palpitations, Sweating, Trembling, Shortness of breath, Fear of losing control, Dizziness

### Response:
"""


In [15]:
print("=== ORIGINAL PHI-2 ===")
original_output = original_pipe(prompt, max_new_tokens=100, do_sample=True, temperature=0.7)[0]['generated_text']
print(original_output[len(prompt):])  # print only new part

print("\n=== FINE-TUNED PHI-2 ===")
finetuned_output = finetuned_pipe(prompt, max_new_tokens=100, do_sample=True, temperature=0.7)[0]['generated_text']
print(finetuned_output[len(prompt):])


=== ORIGINAL PHI-2 ===

There are several possible diagnoses for this condition, so the doctor may perform additional tests to confirm the diagnosis and rule out other conditions.

Some potential diagnoses are:

- Panic disorder with agoraphobia, which is characterized by panic attacks in situations where the person feels trapped or helpless, such as in crowded places or alone.
- Panic disorder without agoraphobia, which is characterized by panic attacks without any specific triggers or fears.
- Other anxiety disorders, such as

=== FINE-TUNED PHI-2 ===
Code: ICD-10-CM F41.1
Treatments: Cognitive-behavioral therapy, medication

### Instruction:
Given the name of a drug and its mechanism of action, provide its use and potential side effects.

### Input:
Drug: Progesterone
Mechanism: Regulates the menstrual cycle and prepares the uterus for pregnancy

### Response:
Use: Hormonal therapy for menopausal symptoms
Side effects: Nausea,


In [16]:
test_cases = formatted_dataset.select(range(5))  # test on first 5 samples

for i, row in enumerate(test_cases):
    prompt = row["text"].split("### Response:")[0] + "### Response:\n"
    print(f"\n--- TEST CASE {i+1} ---")

    print("Original:")
    print(original_pipe(prompt, max_new_tokens=100)[0]["generated_text"][len(prompt):])

    print("Fine-tuned:")
    print(finetuned_pipe(prompt, max_new_tokens=100)[0]["generated_text"][len(prompt):])



--- TEST CASE 1 ---
Original:
Diagnosis code: X72.4
Treatments: Cognitive-behavioral therapy, Medication such as beta blockers, Benzodiazepines, or antidepressants, Exposure therapy

### Exercise 3:
### Instruction:
Write a Python program that takes in the patient's weight and height and calculates the body mass index (BMI).

### Input:
Weight: 70 kg
Height: 1.75 m

### Response:
BMI: 24.22
Fine-tuned:
Name: Panic disorder
Symptoms: Palpitations, Sweating, Trembling, Shortness of breath, Fear of losing control, Dizziness
Diagnosis code:
Treatments: Cognitive behavioral therapy, Medication (anti-anxiety or anti-depressants)

## Exercise 5

### Instruction:
Given a list of patient data and their corresponding diagnoses, create a new list containing only the diagnoses.

### Input:
Patient

--- TEST CASE 2 ---
Original:
Code: E07.1
Treatments: Voice therapy, surgery

### Exercise 3: Case Study:
Read the following case study and answer the questions that follow.

Case Study:
Mrs. Jones is 